In [ ]:
import matplotlib.patches as mpatches
import numpy as np
import os
import math
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
params = {
   'axes.labelsize': 12,
   'font.size': 10,
   'legend.fontsize': 11,
   'xtick.labelsize': 10,
   'ytick.labelsize': 12,
   'text.usetex': False,
   'figure.figsize': [10, 2] # instead of 4.5, 4.5
   }
plt.rcParams.update(params)

## helper function

In [ ]:
def autolabel(rects, ses):
    # attach some text labels
    for rect, se in zip(rects, ses):
        height = rect.get_height()
        plt.axes().text(rect.get_x()+rect.get_width()/2., height+1.03*se, '{:2.3f}'.format(float(height)).lstrip('0'),
                ha='center', va='bottom',fontsize=10)

## Fixed Parameter

In [ ]:
# experiment
exp = 'mysseg'
exp_label = 'Time Segment Matching'
base_md = 'avg' #baseline model
main_md = 'multi_srm' #main model

# dictionaries
cl_dict = {0:1/225,1:2/297,2:2/297,3:2/1973} # chance level of each dataset
feat_dict = {'multi_srm':[75,75,100],'all_srm':[75,75,50],'indv_srm':[75,75,50],'all_ica':[50,25,25],'indv_ica':[50,25,25],\
'all_gica':[100,50,25],'indv_gica':[100,50,25],'all_dict':[25,25,25],'indv_dict':[25,25,25]}
roi_dict = {'dmn':0,'pt':1,'eac':2}
# change multi_srm name!
md_dict = {'avg':'MNI','multi_srm':'MDSRM','ica':'ICA','gica':'Group-ICA','dict':'Dict','srm':'SRM'}
ds_dict = {0:'GreenEyes',1:'Milky',2:'Vodka',3:'Sherlock'}
pre_dict = {'indv_':'Single','all_':'All'}
md_pre = ['indv_','all_']

# paths
input_file = '../../output/accu_bar/{}/{}/{}_ds{}.npz' #exp,roi,model,ds
output_path = '../../output/figures/{}/' #exp
if not os.path.exists(output_path.format(exp)):
    os.makedirs(output_path.format(exp))
output_file = output_path+'{}_ds{}' #roi,ds

## Plotting Parameter

In [ ]:
# max_accu = 0.42
# datasets = [1,2]
model_all = ['ica','gica','dict','srm']
roi_all = ['dmn','pt','eac']
max_accu = 0.6
datasets = [0,3]

## Aggregate Accuracies

In [ ]:
all_mean = [] # length # of roi
all_se = [] # length # of roi
# aggregate each roi
for roi in roi_all:
    roi_mean = []
    roi_se = []
    for ds in datasets:
        # baseline model
        ws=np.load(input_file.format(exp,roi,base_md,ds))
        roi_mean.append(ws['mean'].item())
        roi_se.append(ws['se'].item())
        # add comparison models
        for model in model_all: 
            for pre in md_pre:
                ws=np.load(input_file.format(exp,roi,pre+model,ds))
                roi_mean.append(ws['mean'].item())
                roi_se.append(ws['se'].item())
        # main model
        # indv version: indv srm
        ws=np.load(input_file.format(exp,roi,'indv_srm',ds))
        roi_mean.append(ws['mean'].item())
        roi_se.append(ws['se'].item())
        # main version
        ws=np.load(input_file.format(exp,roi,main_md,ds))
        roi_mean.append(ws['mean'].item())
        roi_se.append(ws['se'].item())
    all_mean.append(roi_mean)
    all_se.append(roi_se)

## Plot all roi

In [ ]:
width=1
nmodel = (len(model_all)+1)*len(md_pre)+1 # number of models, including baseline model and main model
ndata = len(datasets)
group_width = 1.25*width*nmodel
center_all = np.linspace(0,group_width*(ndata-1),ndata)
# set colors
base_color = 'lightgrey'
main_color = 'red'
# at most 8 methods to compare
color_all = ['mediumseagreen','mediumseagreen','dodgerblue','dodgerblue','mediumorchid','mediumorchid','gold','gold']
pattern = ['///',' ']

for m,roi in enumerate(roi_all):
    ds_idx = []
    ds_name = []
    plt.figure()
    # configure bars
    for i,(ds, center) in enumerate(zip(datasets,center_all)):
        mean = list(all_mean[m][i*nmodel:(i+1)*nmodel])
        se   = list(all_se[m][i*nmodel:(i+1)*nmodel])    
        idx = np.arange(center,center+nmodel*width,width)
        error_config = {'ecolor': '0','capsize':3}   
        rects = plt.bar(idx, mean, yerr=se, align='center', error_kw=error_config, width = width-0.1)
        # set colors
        rects[0].set_color(base_color)
        for rect_i in range(1,len(rects)-2):
            rects[rect_i].set_color(color_all[rect_i-1])
        rects[-2].set_color(main_color)
        rects[-1].set_color(main_color)
        # add patterns
        for rect_i in range(1,len(rects),2):
            rects[rect_i].set_hatch(pattern[0])
        autolabel(rects, se)
        ds_idx.append(center+(nmodel-1)*width/2)        
        ds_name.append(ds_dict[ds])
    # find xtick positions
    grp = len(md_pre)
    xtick_idx = []
    xtick_name = []
    for center in center_all:
        # position of bars
        idx = np.arange(center,center+nmodel*width,width)        
        xtick_idx.append(idx[0]) # base
        xtick_name.append(md_dict[base_md])
        for m,model in enumerate(model_all):
            xtick_idx.append(idx[0]+m*grp+1.5*width)
            feat = str(feat_dict[md_pre[0]+model][roi_dict[roi]])
            xtick_name.append(md_dict[model]+'\n k='+feat)
        xtick_idx.append(idx[-1]-0.5*width) # main
        feat = str(feat_dict[main_md][roi_dict[roi]])
        xtick_name.append(md_dict[main_md]+'\n k='+feat)                       
    plt.xticks(xtick_idx,xtick_name)
    
    plt.ylabel('Accuracy')
    left_lim = center_all[0]-0.5*width-0.5
    right_lim = center_all[-1]+(nmodel-0.5)*width+0.5
    plt.xlim([left_lim,right_lim])
    plt.ylim([0,max_accu])

    # plot chance accuracy of each dataset
    for d,center in enumerate(center_all):
        cl = cl_dict[datasets[d]]
        line = plt.plot([center-width, center+(nmodel+1)*width], [cl, cl], 'k-.', linewidth=2)
       
    # Add texts
    plt.text((right_lim+left_lim)/2, 1.2*max_accu, exp_label+', ROI: '+roi.upper(),fontsize=12,horizontalalignment='center', verticalalignment='bottom')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    for d in range(len(datasets)):
        plt.text(ds_idx[d],0.96*max_accu,ds_name[d],bbox=props,fontsize=12,horizontalalignment='center', verticalalignment='top')
    
    # legend
    legend_handle = []
    legend_handle.append(mpatches.Patch(color=base_color, label=md_dict[base_md]))
    for patch_i in range(len(model_all)):
        legend_handle.append(mpatches.Patch(color=color_all[2*patch_i+1], label=md_dict[model_all[patch_i]]))
    legend_handle.append(mpatches.Patch(color=main_color, label=md_dict[main_md]))        
    l1 = plt.legend(handles=legend_handle,bbox_to_anchor=(0.,1.02,1.,.102), loc=3,ncol=len(model_all)+2,mode="expand", borderaxespad=0.)
    ax = plt.gca().add_artist(l1)
    l2 = plt.legend(line ,['chance'],loc=9,ncol=1)
    legend_handle2 = []
    for patch_i in range(len(md_pre)):
        legend_handle2.append(mpatches.Patch(hatch=pattern[patch_i], color='beige',label=pre_dict[md_pre[patch_i]]))
    ax = plt.gca().add_artist(l2)
    plt.legend(handles=legend_handle2,loc=2,ncol=1)
            
#     plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.eps', format='eps', dpi=200,bbox_inches='tight')
    plt.savefig(output_file.format(exp,roi,datasets).replace(' ','')+'.pdf', format='pdf', dpi=200,bbox_inches='tight')